In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 2
Setting new number of workers to: 2
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 2880
Session ID: 4d5b1bc2-cae9-47f4-b9a1-4061cd52f354
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
Waiting for session 4d5b1bc2-cae9-47f4-b9a1-4061cd52f354 to get into ready status...
Session 4d5b1bc2-cae9-47f4-b9a1-4061cd52f354 has been created.



In [10]:
s3_path = 's3://tollywood-api-data-extract-ravi/raw/to_processed/'
source_dyf = glueContext.create_dynamic_frame_from_options(
    connection_type="s3",
    connection_options ={"paths": [s3_path]},
    format="json"
)

In [10]:
# source_dyf.show()

In [12]:
# toDF() will basically convert the entire JSON to dataframe 
spotify_df = source_dyf.toDF()

In [13]:
# spotify_df.show()

+--------------------+--------------------+-----+----+------+--------+-----+
|                href|               items|limit|next|offset|previous|total|
+--------------------+--------------------+-----+----+------+--------+-----+
|https://api.spoti...|[{2021-12-06T12:1...|  100|NULL|     0|    NULL|   40|
+--------------------+--------------------+-----+----+------+--------+-----+


In [14]:
from pyspark.sql.functions import explode, col

In [15]:
spotify_df = spotify_df.withColumn("items", explode("items"))

In [17]:
# spotify_df.show()

+--------------------+--------------------+-----+----+------+--------+-----+
|                href|               items|limit|next|offset|previous|total|
+--------------------+--------------------+-----+----+------+--------+-----+
|https://api.spoti...|{2021-12-06T12:18...|  100|NULL|     0|    NULL|   40|
|https://api.spoti...|{2021-12-06T12:20...|  100|NULL|     0|    NULL|   40|
|https://api.spoti...|{2021-12-06T12:23...|  100|NULL|     0|    NULL|   40|
|https://api.spoti...|{2021-12-06T12:24...|  100|NULL|     0|    NULL|   40|
|https://api.spoti...|{2021-12-06T12:24...|  100|NULL|     0|    NULL|   40|
|https://api.spoti...|{2021-12-06T12:24...|  100|NULL|     0|    NULL|   40|
|https://api.spoti...|{2021-12-06T12:25...|  100|NULL|     0|    NULL|   40|
|https://api.spoti...|{2021-12-06T12:25...|  100|NULL|     0|    NULL|   40|
|https://api.spoti...|{2021-12-06T13:21...|  100|NULL|     0|    NULL|   40|
|https://api.spoti...|{2021-12-06T13:22...|  100|NULL|     0|    NULL|   40|

In [23]:
# tollywood_album_df =spotify_df.select(col("items.track.album.id").alias("album_id") \
#                  ,col("items.track.album.name").alias("album_name") \
#                  ,col("items.track.album.release_date").alias("release_date") \
#                  ,col("items.track.album.external_urls.spotify").alias("album_url") \
#                  ,col("items.track.album.total_tracks").alias("album_total_tracks")).drop_duplicates(['album_id'])

# tollywood_album_df.show()

+--------------------+--------------------+------------+--------------------+------------------+
|            album_id|          album_name|release_date|           album_url|album_total_tracks|
+--------------------+--------------------+------------+--------------------+------------------+
|0AarnnkRN01laay1j...| Chinnadana Neekosam|  2014-11-27|https://open.spot...|                 6|
|0RtAAZvxDk5jXmhys...|               Julai|  2012-06-09|https://open.spot...|                 6|
|0UKhCWkRL6X4Kwbjz...|Awaara (Original ...|  2010-04-10|https://open.spot...|                 6|
|0vHE0cmA8orbBgzIW...|              Uppena|  2020-03-02|https://open.spot...|                 1|
|220pe6dreuJ8550vt...|               Jayam|  2002-05-14|https://open.spot...|                12|
|2dnfny8QAiGbUk9NI...|Ala Vaikunthapurr...|  2019-12-24|https://open.spot...|                 1|
|33thvrIEnv8XgCjul...|          Mr Perfect|  2011-03-18|https://open.spot...|                 9|
|3GP0G5V3s6bEdXCZS...|        

In [61]:
# tollywood_artist_df = spotify_df.select(explode(col("items.track.artists.id")).alias("artist_id") \
#     ,explode(col("items.track.artists.name")).alias("artist_name") \
#     ,explode(col("items.track.artists.external_urls.spotify")).alias("artist_url") \
#     ,col("items.track.album.id").alias("artist_ablum_id")).drop_duplicates(['artist_id']).drop_duplicates(['artist_id'])

# tollywood_artist_df.show()

+--------------------+-----------------+--------------------+--------------------+
|           artist_id|      artist_name|          artist_url|     artist_ablum_id|
+--------------------+-----------------+--------------------+--------------------+
|00hL0oJD4pWef0kPY...|    R. P. Patnaik|https://open.spot...|220pe6dreuJ8550vt...|
|0LSPREIgGMZXCuKVe...|          Karthik|https://open.spot...|4pOqsafnYYQqZcmbf...|
|0PykiHCcDsMPRV7s6...|            Tippu|https://open.spot...|0RtAAZvxDk5jXmhys...|
|0WiZi3Q419nMpAQEq...|  Mickey J. Meyer|https://open.spot...|4nbsEScX6MTTXwXFU...|
|0kJO65h553i1iGsZu...|       Adnan Sami|https://open.spot...|0RtAAZvxDk5jXmhys...|
|0kNlj2vZ0kH8CAKKl...|   Ranjith Govind|https://open.spot...|3GP0G5V3s6bEdXCZS...|
|0oOet2f43PA68X5Rx...|          Karthik|https://open.spot...|4pOqsafnYYQqZcmbf...|
|0scEW6ZihUHK7KtWb...|     Anuj Gurwara|https://open.spot...|7GOB3RBsULP8usndI...|
|12l1SqSNsg2mI2IcX...| M. M. Keeravaani|https://open.spot...|7GOB3RBsULP8usndI...|
|1BI

In [62]:
# Lets create an function for modular programming approach 
def process_albums(df):
    df = df.select(col("items.track.album.id").alias("album_id") \
                 ,col("items.track.album.name").alias("album_name") \
                 ,col("items.track.album.release_date").alias("release_date") \
                 ,col("items.track.album.external_urls.spotify").alias("album_url") \
                 ,col("items.track.album.total_tracks").alias("album_total_tracks")).drop_duplicates(['album_id'])
    return df

def process_artists(df):
    df = df.select(explode(col("items.track.artists.id")).alias("artist_id") \
    ,explode(col("items.track.artists.name")).alias("artist_name") \
    ,explode(col("items.track.artists.external_urls.spotify")).alias("artist_url") \
    ,col("items.track.album.id").alias("artist_ablum_id")).drop_duplicates(['artist_id']).drop_duplicates(['artist_id'])
    
    return df

In [34]:
tollywood_album_df = process_albums(spotify_df)
tollywood_album_df.show()

+--------------------+--------------------+------------+--------------------+------------------+
|            album_id|          album_name|release_date|           album_url|album_total_tracks|
+--------------------+--------------------+------------+--------------------+------------------+
|0AarnnkRN01laay1j...| Chinnadana Neekosam|  2014-11-27|https://open.spot...|                 6|
|0RtAAZvxDk5jXmhys...|               Julai|  2012-06-09|https://open.spot...|                 6|
|0UKhCWkRL6X4Kwbjz...|Awaara (Original ...|  2010-04-10|https://open.spot...|                 6|
|0vHE0cmA8orbBgzIW...|              Uppena|  2020-03-02|https://open.spot...|                 1|
|220pe6dreuJ8550vt...|               Jayam|  2002-05-14|https://open.spot...|                12|
|2dnfny8QAiGbUk9NI...|Ala Vaikunthapurr...|  2019-12-24|https://open.spot...|                 1|
|33thvrIEnv8XgCjul...|          Mr Perfect|  2011-03-18|https://open.spot...|                 9|
|3GP0G5V3s6bEdXCZS...|        

In [58]:
tollywood_artist_df = process_artists(spotify_df)
tollywood_artist_df.show()

AssertionError: 


In [64]:
def write_to_s3(df, path_suffix, format_type="csv"):
    # Convert back to DynamicFrame
    dynamic_frame = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")
    
    glueContext.write_dynamic_frame.from_options(
        frame = dynamic_frame,
        connection_type = "s3",
        connection_options = {"path": f"s3://tollywood-api-data-extract-ravi/transformed/{path_suffix}/"},
        format = format_type
    )

In [65]:
from datetime import datetime
write_to_s3(tollywood_album_df, 'album_transformed/album_transformed_{}'.format(datetime.now()))

In [66]:
write_to_s3(tollywood_artist_df, 'artist_transformed/artist_transformed_{}'.format(datetime.now()))